<a href="https://colab.research.google.com/github/mshinko/Research-project/blob/main/Independent_Study2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install datasets transformers rouge-score nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 113.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.9 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=85289bb2cbd1808e2dde08883ec5eb3b1c32f7bf0819e16c5f17df3a6d75c6d0
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [2]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 7.9 MB/s eta 0:00:00


In [3]:
pip install accelerate -u


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


In [4]:
import transformers

print(transformers.__version__)

4.33.2


In [5]:
model_checkpoint = "t5-small"

In [6]:
pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.0 MB/s eta 0:00:00


In [7]:
pip install jsonlines

In [86]:
import jsonlines
import pandas as pd
from google.colab import files

# Use triple-quoted strings to preserve the formatting
original_text = """
  To: VRSS Management Team
    From: David Schlingloff
    Subject: VRSS Management Report – Week ending January 30th, 2022
    Copies: Management Team, Advisors

    Priorities
    We are working on these priorities:
    1. Finalize a set of interested and motivated team members.
    2. Define competitors to the VRSS product.
    3. Identify possible methods of VR streaming.

    Progress
    During this (short) reporting period, we accomplished these tasks:
    - Developed the idea and company:
      - Brainstormed possible ideas to pursue and decided upon the VR service.
      - Created a name for the company (Virtual Reality Sports Seating - VRSS).
    - Founded the Team:
      - Confirmed 4 teammates (we may accept more in the following days).
      - Drafted our MOKRs for discussion.

    Plans
    During the next week, we plan to finish these tasks:
    - Find 5 competitors to the planned VRSS service:
      - Perform research online to develop 5 possible competitors (both direct and indirect) to VRSS service by Friday, February 4th (Alex, Ryan, Matt).
    - Brainstorm 4 VR streaming-capable cameras:
      - Find 4 different types of video cameras (preferably inexpensive, <$1000) that can support VR streaming capabilities by Friday, February 4th.
      - Begin understanding the limitations of streaming bandwidth (David, Kartik).

    Problems
    We are aware of these problems:
    - We need to finalize our team:
      - There are still people who were interested in joining last week but left too early to make a final decision. We would like to complete this by the end of class on Monday, January 31st.
    - We are unsure if cameras can support our bandwidth:
      - The team would like to use a limited number of cameras that can be accessed by many different people during a game. The team is unsure if it is even possible to get these types of cameras without spending excessive amounts of money.
    - We do not currently have experience with software/website creation:
      - Much of the VRSS streaming service is dependent on a reliable stream with easy website access. A lot of our resources will need to be spent on attempting to create an appealing website with streaming capabilities.

    Management Report.doc 2022Jan30 Page 1/2

    Attachments
    - None.
"""

# Define the summary
summary = "Week 1 Created the team and MOKR. Plan to find competitors and understand technology needed. They lack the knowledge to build a streaming service solution and it is unclear who their customers are that would pay for it. Identified problems but don’t have plans for mitigating."


# Create a DataFrame
df = pd.DataFrame({'Original Text': [original_text], 'Summary': [summary]})
# Define the new row data as a dictionary
new_row = {
    'Original Text': """
    To: VRSS Management Team
    From: David Schlingloff
    Subject: VRSS Management Report – Week ending February 6th, 2022
    Copies: Management Team, Advisors

    Priorities
    We are working on these priorities:
    1. Finalize a set of interested and motivated team members.
    2. Defining competitors to the VRSS product.
    3. Identifying possible methods of VR streaming.

    Progress
    During this (short) reporting period, we accomplished these tasks:
    - Find 5 competitors to the planned VRSS service – Developed 5 possible
    competitors (both direct and indirect) to VRSS service.
    - Brainstorm 4 VR streaming capable cameras - Found 4+ different types of
    video cameras (preferably inexpensive, <$1000) that can support VR
    streaming capabilities.

    Plans
    During the next week, we plan to finish these tasks:
    - Refine our target consumers - Talk about refining the consumers we defined
    last week. Build upon this by creating 3 theoretical customer profiles (David,
    Alex, Matthew).
    - Define our advisory board - We will think about the people that would be
    helpful for us to talk to. Think about areas we could ask questions in. Using
    these topics, search for people we can possibly talk to and discuss our project
    plan. (Ryan, Entire Team).
    - Find specific details about 3 competitors - From the competitors we found
    last week, go on their websites and explore what they do and learn about
    their history. Specifically, create a document by February 11th, with the
    following 3 things noted for each: similarities/differences between company
    and VRSS, the prices for that service, and if/how they advertise. (Kartik,
    Ryan).

    Problems
    We are aware of these problems:
    - We do not currently have experience with software / website creation –
    Much of the VRSS streaming service is dependent on a reliable stream
    with easy website access. A lot of our resources will need to be spent on
    attempting to create an appealing website with streaming capabilities.
    - Defining our place in the market is tricky - The team mapped out the
    two existing markets - going to sports games and streaming them on TV.
    We realized that there is a definitive space in the market for VR streaming,
    but it might be tricky to convince the target consumers to pursue this
    option versus the others.
    - Finding VR capable equipment is expensive - VR is still a relatively new
    type of streaming, and therefore the market for this technology is still
    relatively niche. Finding equipment (like a VR capable camera) is possible,
    but our requirement of <$1000 makes it difficult.
    - Our marketing efforts/strategy is a current problem we have. In order to
    acquire a lot of customers and produce sales, our marketing and
    advertising will need to be phenomenal. We could develop a social media
    account (Instagram/Facebook) and/or pay for a commercial
    advertisement on TV.

    Management Report.doc 2022Feb06 Page 1/5

    Attachments
    - None.
    """,
    'Summary': """
    Week 2
    Found competitors and VR streaming cameras. Plan on analyzing competition, find advisory board members, and create 3 customer profiles. They lack the knowledge to build a streaming service solution, the equipment is too expensive and it is unclear who their customers are that would pay for it. Identified problems but don’t have plans for mitigating.
    """
}

# Append the new row to the DataFrame
df = df.append(new_row, ignore_index=True)
# Define the new row
new_row_2 = {
    'Original Text': """
   To: VRSS Management Team
From: David Schlingloff
Subject: VRSS Management Report – Week ending February 13th, 2022
Copies: Management Team, Advisors
Priorities
We are working on these priorities:
1. Gauge the interest of VR equipment with our target markets
2. Identify a way to live-stream in VR
3. Brainstorm ways to begin website creation
Progress
During this reporting period, we accomplished these tasks:
● Refined our target consumers - Developed our ideal target market by
creating 3 theoretical customer profiles based on the 3 customer groups that
the team developed.
● Defined our advisory board - Listed nine people that the team would like to
invite to the first project review meeting. Each person has different unique
talents and abilities that could help the team move forward.
● Scheduled VR equipment rental - Reserved a VR camera and VR headset at
the Virginia Tech library so that the team can begin testing with the
equipment.
● Find specific details about 3 competitors - Unable to accomplish this in the
week timeframe. The team decided that the information we already have is
enough to move forward with and we spent our resources on other tasks.
Plans
During the next week, we plan to finish these tasks:
● Create an interest survey - Use google forms to develop a survey that can
gauge the interest in this service based on the 3 target markets defined by
February 17th. Generate a list of 9 people (three from each target market) to
be sent out by February 21st. (Kartik, Ryan, Alex)
● Obtain and use VR equipment - Receive the VR camera and headset from
the Virginia Tech library and attempt to connect to the computer. See if it is
possible to first record a video in VR. Then work on streaming options if time
permits for the week. The team will research and document the process for
video recording and video streaming (if possible). (David, all team)

Management Report.doc 2022Feb13 Page 1/3

Problems
We are aware of these problems:
● We still do not currently have experience with software / website
creation – Much of the VRSS streaming service is dependent on a reliable
stream with easy website access. A lot of our resources will need to be
spent on attempting to create an appealing website with streaming
capabilities. However, we are now looking at meeting with people who do
have this experience and developing a plan of integration.
● VR equipment implementation requires resources - Although the team
was successful in figuring out a way to rent VR equipment, it has realized
that in order to record and stream video, a powerful and expensive
computer is needed. It will need to be portable so that it can be moved to
where the VR cameras are located.
● Our marketing efforts/strategy - In order to acquire a lot of customers and
produce sales, our marketing and advertising will need to be phenomenal.
We could develop a social media account (instagram/facebook) and/or
pay for
    """,
    'Summary': """
    Week 3
    Created customer profiles, list of potential advisory board members, and reserved VR equipment rentals to address the expense constraint identified last week. Failed to analyze competitors they believe they have sufficient information to move forward though not clear what that is. They plan to create an interest survey (as opposed to interviewing as indicated in MOKR) and record a VR video with rented equipment. They lack the knowledge to build a streaming service solution, the equipment is too expensive. They continue to have the same problems but don’t have plans for mitigating. Will need to intervene to get them unstuck.
    """
}


# Append the new row to the DataFrame
df = df.append(new_row_2, ignore_index=True)
new_row3 = {
    'Original Text': """
    To: VRSS Management Team
From: David Schlingloff
Subject: VRSS Management Report – Week ending March 4th, 2022
Copies: Management Team, Advisors
Priorities
We are working on these priorities:
1. Get people to take our survey and evaluate the results.
2. Identify a way to live-stream in VR
3. Submit a video to KickstartVT
Progress
During this reporting period, we accomplished these tasks:
● Analyzed interest survey results - Received 25 responses from the survey and
wrote a one-page report summarizing the key-findings from the survey that was
sent out. The team found that we should continue to pursue this product. (Matt,
Ryan, Alex)
● Obtained and use VR equipment - Obtained the Oculus Go and successfully
connected it to YouTube VR. The process was documented and an Oculus Quest
2 was scheduled for rent to attempt to do the same thing. If it works as well, then
we will discuss using the Insta360 camera to live-stream to the Oculus Quest 2.
(David, Kartik)
● Performed individual project reviews - Gave the presentation to our advisory
board members. (David, all team members)

Plans
During the next week, we plan to finish these tasks:
● Finalize interest survey results - Synthesize the survey results and add to our
report and project review presentation. (Ryan, Matt, Alex)
● Obtain and use VR equipment - Obtain the Oculus Quest 2 and see if it can
connect to YouTube VR. If so, document the process and start to discuss
live-streaming options using the Insta360 camera. (Alex, Kartik)
● Perform team project review - Revise presentation template for the class and
present the project review to the team on Wednesday, March 16th. Record the
comments from the Q&A. (David, all team members)
● Create a video for KickStartVT - Record and edit a video for KickstartVT and
submit on Sunday, March 20th. (David, all team members)

Problems
We are aware of these problems:
● We can only rent equipment for a limited period - We would like to rent an
Oculus Go or an Oculus Quest (since they are stand-alone devices), but they
require rentals so timing can get delayed significantly.
● We still do not currently have experience with software / website creation –
Much of the VRSS streaming service is dependent on a reliable stream with easy
website access. A lot of our resources will need to be spent on attempting to
create an appealing website with streaming capabilities. However, we are now
looking at meeting with people who do have this experience and developing a
plan of integration.
● Our marketing efforts/strategy - In order to acquire a lot of customers and
produce sales, our marketing and advertising will need to be aggressive. We
could develop a social media account (instagram/facebook) and/or pay for a
commercial advertisement on TV.
    """,
    'Summary': """
  Week 5
Sent out the previously created interest survey and tried out an HTC Vive but didn’t  function with Youtube App. Scheduled time with advisory board and potential team member. Plan to try an Oculus Go, analyze the survey data, and prepare for the project review. They still lack the knowledge to build a streaming service solution, the renting of the vr equipment is too burdensome  due to rental policies. They continue to have the same problems of website creation and marketing  but don’t have plans for mitigating. Will need to intervene to get them unstuck.


    """
}
df = df.append(new_row3, ignore_index=True)
# Print the updated DataFrame
print(df)




                                       Original Text  \
0  \n  To: VRSS Management Team\n    From: David ...   
1  \n    To: VRSS Management Team\n    From: Davi...   
2  \n   To: VRSS Management Team\nFrom: David Sch...   
3  \n    To: VRSS Management Team\nFrom: David Sc...   

                                             Summary  
0  Week 1 Created the team and MOKR. Plan to find...  
1  \n    Week 2\n    Found competitors and VR str...  
2  \n    Week 3\n    Created customer profiles, l...  
3  \n  Week 5\nSent out the previously created in...  


<ipython-input-86-f6790bda8678>:127: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
<ipython-input-86-f6790bda8678>:194: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row_2, ignore_index=True)
<ipython-input-86-f6790bda8678>:256: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row3, ignore_index=True)


In [87]:
from datasets import load_dataset
import evaluate

metric = evaluate.load("rouge")

In [88]:
df['Original Text'][0]

'\n  To: VRSS Management Team\n    From: David Schlingloff\n    Subject: VRSS Management Report – Week ending January 30th, 2022\n    Copies: Management Team, Advisors\n\n    Priorities\n    We are working on these priorities:\n    1. Finalize a set of interested and motivated team members.\n    2. Define competitors to the VRSS product.\n    3. Identify possible methods of VR streaming.\n\n    Progress\n    During this (short) reporting period, we accomplished these tasks:\n    - Developed the idea and company:\n      - Brainstormed possible ideas to pursue and decided upon the VR service.\n      - Created a name for the company (Virtual Reality Sports Seating - VRSS).\n    - Founded the Team:\n      - Confirmed 4 teammates (we may accept more in the following days).\n      - Drafted our MOKRs for discussion.\n\n    Plans\n    During the next week, we plan to finish these tasks:\n    - Find 5 competitors to the planned VRSS service:\n      - Perform research online to develop 5 possib

In [89]:
metric

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [90]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [91]:
if model_checkpoint in ["google/t5-small-lm-adapt", "t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
  prefix = "summarize: "
else:
  prefix =""

In [92]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(row):
    doc = row["Original Text"]
    summary = row["Summary"]

    inputs = [prefix + doc]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(summary, max_length=max_target_length, truncation=True, padding=True)

    # Flatten the input_ids and attention_mask
    model_inputs = {key: [item for sublist in value for item in sublist] for key, value in model_inputs.items()}

    # Flatten the labels
    labels = [item for item in labels["input_ids"]]

    # Return a dictionary containing the model inputs and labels
    return {
        "input_ids": model_inputs["input_ids"],
        "attention_mask": model_inputs["attention_mask"],
        "labels": labels,
    }

In [93]:
# Apply the preprocess_function to your DataFrame
preprocessed_data = df.apply(preprocess_function, axis=1)
# Print the preprocessed data
print(preprocessed_data)

0    {'input_ids': [21603, 10, 304, 10, 13893, 4256...
1    {'input_ids': [21603, 10, 304, 10, 13893, 4256...
2    {'input_ids': [21603, 10, 304, 10, 13893, 4256...
3    {'input_ids': [21603, 10, 304, 10, 13893, 4256...
dtype: object


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [94]:
from sklearn.model_selection import train_test_split

# Split your DataFrame into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Apply the preprocess_function to both train and validation DataFrames
train_tokenized = train_df.apply(preprocess_function, axis=1).tolist()
val_tokenized = val_df.apply(preprocess_function, axis=1).tolist()

# Create dictionary-based datasets
tokenized_datasets = {'train': train_tokenized, 'validation': val_tokenized}

# Print the tokenized datasets
print(tokenized_datasets)

{'train': [{'input_ids': [21603, 10, 304, 10, 13893, 4256, 2159, 2271, 1029, 10, 1955, 13675, 53, 40, 1647, 19237, 10, 13893, 4256, 2159, 3750, 3, 104, 6551, 7784, 1332, 314, 189, 6, 460, 2884, 638, 8497, 10, 2159, 2271, 6, 17228, 7, 6783, 2197, 101, 33, 464, 30, 175, 12697, 10, 1300, 1609, 151, 12, 240, 69, 3719, 11, 6825, 8, 772, 5, 1682, 3, 23393, 3, 9, 194, 12, 619, 18, 8103, 16, 13893, 1877, 3325, 1538, 3, 9, 671, 12, 20759, 10208, 18645, 24583, 3, 2092, 48, 5099, 1059, 6, 62, 10380, 175, 4145, 10, 3, 2, 5331, 120, 776, 26, 1046, 3719, 772, 3, 18, 24083, 26, 944, 7216, 45, 8, 3719, 11, 2832, 3, 9, 80, 18, 6492, 934, 4505, 1635, 2610, 8, 843, 18, 8954, 53, 7, 45, 8, 3719, 24, 47, 1622, 91, 5, 37, 372, 435, 24, 62, 225, 916, 12, 6665, 48, 556, 5, 41, 329, 144, 17, 6, 7826, 6, 5104, 61, 3, 2, 4249, 10733, 11, 169, 13893, 1277, 3, 18, 4249, 10733, 8, 411, 1497, 302, 1263, 11, 4234, 2895, 34, 12, 5343, 13893, 5, 37, 433, 47, 15552, 11, 46, 411, 1497, 302, 2415, 222, 204, 47, 5018, 21, 

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [95]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [96]:
from transformers import Seq2SeqTrainingArguments

batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy="steps",  # Change this line to use "steps" for evaluation strategy
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=3,
    load_best_model_at_end=True,
    save_steps=500,
    eval_steps=500,
    push_to_hub=False,
    logging_steps=100,
    report_to="tensorboard",
)

In [97]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [98]:
import nltk
import numpy as np
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
  decoed_labels = ["\n".join(nltk.sent_tokenizer(label.strip())) for label in decoded_labels]

  result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

  result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

  predictions_lens = [np.count_nonzero(pred != tokenizer.pad_tokenid)for pred in predictions]
  result['gen_len'] = np.mean(predictions_lens)

  return {k: round(v, 4) for k, v in result.items()}

In [99]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [100]:

trainer.train()


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=3, training_loss=5.639340082804362, metrics={'train_runtime': 21.3624, 'train_samples_per_second': 0.421, 'train_steps_per_second': 0.14, 'total_flos': 1601104084992.0, 'train_loss': 5.639340082804362, 'epoch': 3.0})

In [83]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [101]:
# Load the trained model and tokenizer
model_checkpoint = "t5-small"  # Replace with your model checkpoint
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Input text
input_text = """
    To: VRSS Management Team
From: David Schlingloff
Subject: VRSS Management Report – Week ending March 21st, 2022
Copies: Management Team, Advisors
Priorities
We are working on these priorities:
1. Get more people to take our survey and evaluate the results, possibly older folks.
2. Plan if we would like to rent out equipment and use it at an intramural game on
April 1st.
3. Change/ review future plans based on project review feedback.
Progress
During this reporting period, we accomplished these tasks:
● Finalized interest survey results - Synthesized the survey results and added to
our report and project review presentation. (Ryan, Matt, Alex)
● Obtained and used VR equipment - Obtained the Oculus Quest 2 and
connected to YouTube VR.
● Performed team project review - Revised presentation template for the class and
presented the project review to the team on Wednesday, March 16th. Recorded
the comments from the Q&A. (David, all team members)
● Created a video for KickStartVT - Recorded and edited a video for KickstartVT
and submitted on Sunday, March 20th. (David, all team members)

Plans
During the next week, we plan to finish these tasks:
● Reach more people with survey -Try and reach more people with our survey
and try to find a way to reach older folks, possibly interviews. Try to get 3+
people for this. (Ryan, Matt, Alex)
● Plan to stream a game at an intramural game- We got invited to an intramural
game to try and put out equipment and streaming capabilities to the test. So we
will plan to see if we can go and rent the equipment and stream. (Kartik, David)
● Review project review comments and plan - Review the comments we received
from the project review and plan / change future plans by updating the MOKR.
(David, all team members)

Problems
We are aware of these problems:
● We can only rent equipment for a limited period - We would like to rent an
Oculus Go or an Oculus Quest (since they are stand-alone devices), but they
require rentals so timing can get delayed significantly.
● We still do not currently have experience with software / website creation –
Much of the VRSS streaming service is dependent on a reliable stream with easy
website access. A lot of our resources will need to be spent on attempting to
create an appealing website with streaming capabilities. However, we are now
looking at meeting with people who do have this experience and developing a
plan of integration.
● Our marketing efforts/strategy - In order to acquire a lot of customers and
produce sales, our marketing and advertising will need to be aggressive. We
could develop a social media account (instagram/facebook) and/or pay for a
commercial advertisement on TV.
"""

# Tokenize and preprocess the input text
inputs = tokenizer("summarize: " + input_text, return_tensors="pt", max_length=1024, truncation=True, padding=True)

# Use the model to generate a summary
summary_ids = model.generate(inputs["input_ids"], max_length=128, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)

# Decode and print the generated summary
generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Generated Summary:")
print(generated_summary)

Generated Summary:
VRSS Management Report – Week ending March 21st, 2022 Copies: Management Team, Advisors Priorities We are working on these priorities: 1. Get more people to take our survey and evaluate the results, possibly older folks. 3. Plan if we would like to rent out equipment and use it at an intramural game on April 1st.
